In [ ]:
import watertap as wt
import pandas as pd
from pyomo.environ import value, Block
from idaes.core import FlowsheetBlock
import numpy as np
from case_study_trains import *
import case_study_trains
import pyomo.environ as env
from pyomo.environ import Constraint

m = wt.watertap_setup(dynamic=False)


wt.case_study_trains.train = {"case_study": "ashkelon",
                             "reference": "nawi",
                             "scenario": "baseline"}


wt.case_study_trains.source_water = {"case_study": "ashkelon", 
                             "reference": "nawi",
                             "scenario": "baseline",
                             "water_type": "seawater"}

m = wt.case_study_trains.get_case_study(m=m)


In [ ]:
wt.display.show_train2(model_name=m)

In [ ]:
### # RUN MODEL with optimal ro --> estimating area and pressure for optimal LCOW
# so that the model solves and gets you results. Then runs again with set pressure.
for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        getattr(m.fs, key).feed.pressure.unfix()
        getattr(m.fs, key).membrane_area.unfix()
        print("Unfixing feed presure and area for", key, '...\n')
        
wt.run_water_tap(m=m, objective=True, print_model_results="summary", skip_small=True)

m = wt.set_bounds(m)

wt.print_ro_results(m)
       

In [ ]:
# If you need the system recovery to match better.... set a maximum recovery rate.
# The previous system recovery results must be greater than the limit you set below. If this is not the case, then
# you need to ease the bounds on the flux assumpations.

desired_recovery = 0.44

if m.fs.costing.system_recovery() > desired_recovery:
    m.fs.recovery_bound = Constraint(expr=m.fs.costing.system_recovery <= desired_recovery)
    
    wt.run_water_tap(m=m, objective=True, print_model_results="summary", skip_small=True)

    wt.print_ro_results(m)

else:
    print("system recovery already lower than desired recovery. desired:", desired_recovery, 
          "current:", m.fs.costing.system_recovery())

In [ ]:
###### RUN MODEL WITH 0 DEGREES OF FREEDOM

In [ ]:
m = wt.run_ro_no_freedom(m)

In [ ]:
###### SENSITIVITY ANALYSES 

In [ ]:
# creates csv in results folder with the name: *case_study*_*scenario*.csv
# In this case, save the final baseline result.

df = wt.get_results_table(m=m, case_study=wt.case_study_trains.source_water["case_study"], 
                                scenario=wt.case_study_trains.source_water["scenario"])

In [ ]:
# # run to rest before sensitivity

# sensitivity
case_study=wt.case_study_trains.source_water["case_study"]
scenario=wt.case_study_trains.source_water["scenario"]
wt.sensitivity_runs(m = m, save_results = True, scenario = scenario, case_study = case_study, skip_small_sens = True)

In [ ]:
import pyomo.util.infeasible as infeas
print(infeas.log_infeasible_bounds(m))
print(infeas.log_infeasible_constraints(m))